In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.672598,"""a""",0.024858,0.173472,0.473333,0.212616,0.301116,-0.654199,0.034028
0.841471,1,"""a""",0,0.725032,"""a""",0.448258,0.369609,0.442519,0.607929,0.788311,-0.485564,0.180443
0.909297,2,"""a""",1,0.125921,"""a""",0.313464,0.817093,0.093865,0.159812,0.64238,0.151433,0.399107
0.14112,3,"""a""",0,0.239125,"""a""",0.160517,0.302076,0.245331,0.277571,0.665437,-0.253199,0.132274
-0.756802,4,"""a""",0,0.159514,"""a""",0.085175,0.817742,0.567512,0.461071,0.124233,-0.593079,0.322391


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.212616,-0.301116
-0.841471,-0.607929,-0.788311
-0.909297,-0.159812,-0.64238
-0.14112,-0.277571,-0.665437
0.756802,-0.248455,0.176884


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.491358, -0.338069]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.491358, -0.338069]","[0.087282, 0.407502]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.491358, -0.338069]","[0.087282, 0.407502]"


In [10]:
pds.config.LIN_REG_EXPR_F64 = True

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.218347,0.001688,129.321148,0.0,0.215037,0.221657,0.994392,0.99439
"""exp(x2)""",0.172665,0.000669,257.952367,0.0,0.171353,0.173977,0.994392,0.99439
"""sin(x3)""",-1.743753,0.001347,-1294.755765,0.0,-1.746393,-1.741113,0.994392,0.99439
"""__bias__""",-0.104271,0.001511,-69.024869,0.0,-0.107232,-0.101309,0.994392,0.99439


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.21835,0.001688,129.323059,0.0,0.215041,0.22166,0.994392,0.99439
"""exp(x2)""",0.172665,0.000669,257.951874,0.0,0.171352,0.173977,0.994392,0.99439
"""sin(x3)""",-1.743752,0.001347,-1294.754517,0.0,-1.746392,-1.741112,0.994392,0.99439
"""__bias__""",-0.104271,0.001511,-69.025009,0.0,-0.107232,-0.101309,0.994392,0.99439


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.506624, -0.327209]"
"""a""","[-0.506624, -0.327209]"
"""a""","[-0.506624, -0.327209]"
"""a""","[-0.506624, -0.327209]"
"""a""","[-0.506624, -0.327209]"
…,…
"""b""","[-0.475911, -0.348862]"
"""b""","[-0.475911, -0.348862]"
"""b""","[-0.475911, -0.348862]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.024858,0.173472,-0.654199,-0.070859,-0.583339
0.448258,0.369609,-0.485564,-0.345209,-0.140355
0.313464,0.817093,0.151433,-0.430257,0.58169
0.160517,0.302076,-0.253199,-0.180994,-0.072205
0.085175,0.817742,-0.593079,-0.318305,-0.274774


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.506624, -0.327209]"
"""b""","[-0.475911, -0.348862]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.340152, -0.157604]"
"""b""","[-0.298948, -0.181451]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.551634
"""a""",-0.542692


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.654199,0.024858,0.173472,null,null
-0.485564,0.448258,0.369609,null,null
0.151433,0.313464,0.817093,null,null
-0.253199,0.160517,0.302076,null,null
-0.593079,0.085175,0.817742,"[-0.191563, -0.392919]",-0.337623
…,…,…,…,…
0.034214,0.183652,0.969768,"[-1.118598, 0.032344]",-0.174067
-0.191267,0.199301,0.871618,"[-1.00787, 0.165373]",-0.056728
-1.173736,0.788381,0.365705,"[-1.371299, 0.24686]",-0.990829


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[28.935657, 28.809545, 28.691804]"


In [22]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.933214,"[0.639196, -0.769044]"
28.752908,"[-0.769044, -0.639196]"


In [23]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.028891
-0.150883
-0.32509
-0.267551
0.265952


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.324568,0.977626,0.508918,0.515456,0.512166,0.504932,0.511972
"""a""",0.336685,1.006825,0.510821,0.48829,0.499302,0.509859,0.494326


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [27]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [28]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [29]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [30]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [31]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [32]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [33]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [34]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.312281
-2.589347
2.107346


In [35]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.540408,null
null,-0.180752,null
-0.312281,0.790719,1.59344
-2.589347,1.816533,1.734638
2.107346,0.909268,0.153466


In [36]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""SjEA""",null
null,"""V6g""",null
-0.312281,"""jLRi""","""TWm6q"""
-2.589347,"""Fox""","""n"""
2.107346,"""ms1pL""","""7fC"""


In [37]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.312281,"""vCAir"""
-2.589347,"""gJY3z"""
2.107346,"""qdQLK"""


In [38]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-1.236769,null,-1.23691
null,-1.378302,null,-1.378317
-0.312281,-1.772519,2.748455,-1.772804
-2.589347,1.926742,2.69087,1.926365
2.107346,-1.99922,2.891562,-1.999712


In [39]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.543997,0.054797,9
null,0.968943,0.759275,6
-0.312281,0.679812,0.556152,3
-2.589347,0.176167,0.903726,9
2.107346,0.014253,0.823778,7


In [40]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.241727,0.809025,5.556779,0.062138


In [41]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.899357,0.007668,1,1
1,0.755517,0.237047,4,3
2,0.67886,0.69925,1,8
0,0.306061,0.106352,2,1
1,0.184794,0.526519,2,3


In [42]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.168437,0.866243}","{37.805485,0.3868}","{0.490655,0.742633}"


In [43]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬─────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test              ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                 ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]           ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪═════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.347426,0.728293} ┆ {39.702883,0.308437} ┆ {0.990061,0.411732} │
│ 1         ┆ {0.279156,0.780142} ┆ {43.453355,0.183641} ┆ {0.618801,0.649145} │
│ 2         ┆ {-0.32765,0.743197} ┆ {48.178912,0.084401} ┆ {1.435433,0.219809} │
└───────────┴─────────────────────┴──────────────────────┴─────────────────────┘


In [44]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
543,0.1086
568,0.1136
566,0.1132
579,0.1158
544,0.1088
539,0.1078
566,0.1132
528,0.1056
567,0.1134


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [45]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [46]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.680468,0.799986,0.636472,0.683598,2.619832,17
1,0.632237,0.009536,0.188717,0.150962,8.932759,7
2,0.568799,0.415959,0.044282,0.855895,5.541483,17
3,0.222506,0.023652,0.258236,0.644228,3.605936,12
4,0.450047,0.586091,0.048578,0.951362,2.900244,14


In [47]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.680468,0.799986,0.636472,0.683598,2.619832,533
1,0.632237,0.009536,0.188717,0.150962,8.932759,3
2,0.568799,0.415959,0.044282,0.855895,5.541483,827
3,0.222506,0.023652,0.258236,0.644228,3.605936,309
4,0.450047,0.586091,0.048578,0.951362,2.900244,999


In [48]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.680468,0.799986,0.636472,0.683598,2.619832,"[0, 1986, … 917]"
1,0.632237,0.009536,0.188717,0.150962,8.932759,"[1, 471, … 835]"
2,0.568799,0.415959,0.044282,0.855895,5.541483,"[2, 299, … 224]"
3,0.222506,0.023652,0.258236,0.644228,3.605936,"[3, 592, … 1571]"
4,0.450047,0.586091,0.048578,0.951362,2.900244,"[4, 625, … 1736]"


In [49]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 1986, … 445] ┆ 8                  │
│ 1   ┆ [1, 471]         ┆ 1                  │
│ 2   ┆ [2, 299, … 369]  ┆ 10                 │
│ 3   ┆ [3, 592, … 399]  ┆ 6                  │
│ 4   ┆ [4, 625, … 1621] ┆ 10                 │
└─────┴──────────────────┴────────────────────┘


In [50]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.680468,0.799986,0.636472,0.683598,2.619832,"[0, 1986, … 917]","[0.0, 0.054245, … 0.068886]"
1,0.632237,0.009536,0.188717,0.150962,8.932759,"[1, 471, … 835]","[0.0, 0.076231, … 0.117019]"
2,0.568799,0.415959,0.044282,0.855895,5.541483,"[2, 299, … 224]","[0.0, 0.028702, … 0.055246]"
3,0.222506,0.023652,0.258236,0.644228,3.605936,"[3, 592, … 1571]","[0.0, 0.007734, … 0.07812]"
4,0.450047,0.586091,0.048578,0.951362,2.900244,"[4, 625, … 1736]","[0.0, 0.0315, … 0.045964]"


In [51]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.680468,0.799986,0.636472,0.683598,2.619832
7,0.178886,0.234666,0.369349,0.450831,9.314647
11,0.593674,0.422257,0.753301,0.094872,1.351538
15,0.394807,0.705486,0.141611,0.696937,4.181464
21,0.529629,0.413993,0.383756,0.030942,0.033844


In [52]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
44,0.528735,0.466712,0.046261,0.181008,6.55354
90,0.493503,0.453868,0.329652,0.983545,2.728804
97,0.486554,0.558675,0.048164,0.800131,8.004868
98,0.541604,0.445024,0.878861,0.996516,2.487449
114,0.490114,0.420707,0.896354,0.891134,0.665363


In [53]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
44,0.528735,0.466712,0.046261,0.181008,6.55354
97,0.486554,0.558675,0.048164,0.800131,8.004868
439,0.512888,0.489822,0.62966,0.274786,9.519738
470,0.54804,0.442806,0.401631,0.050534,8.65371
625,0.456583,0.558575,0.034706,0.631777,8.198693


In [54]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1303, 1534]",3
1,"[1, 1423]",2
2,"[2, 205, … 1221]",6
3,"[3, 487, … 1520]",6
4,"[4, 1937, 1736]",3


# Compatibility

## Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures. 

## Using PDS with Narwhals

Limited

In [55]:
import pandas as pd
import numpy as np
import polars as pl
import polars_ds as pds
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_3918/181413171.py:5: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
0.0,0.022554,0,1,"""ib""","""7"""
1.0,0.968186,0,9,"""FK""","""1O"""
0.0,0.766648,2,3,"""n""","""g"""
1.0,0.526669,1,5,"""tX""","""PX"""
1.0,0.341943,2,6,"""J""","""fB"""


In [56]:
df_pd = df.to_pandas()

In [57]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

0    0.3
Name: , dtype: float64

In [58]:
# Polars Series
print(pds2.query_roc_auc(df["actual"], df["predicted"]))
# NumPy
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_numpy(), df["predicted"].to_numpy()))
pds2.return_numpy = False
# Pandas
print(pds2.query_roc_auc(df["actual"].to_pandas(), df["predicted"].to_pandas()))
# PyArrow
# Arrow series can be inputs, but the output cannot be converted correctly. Please let me know if you have a fix.
# The work around is to use NumPy for Arrow
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_arrow(), df["predicted"].to_arrow()))
# Other array-protocal compatible inputs
# print(pds2.query_roc_auc(df["actual"].to_jax(), df["predicted"].to_jax()))

pds2.return_numpy = False


shape: (1,)
Series: '' [f64]
[
	0.501332
]
[0.50133184]
0    0.501332
Name: , dtype: float64
[0.50133184]


In [59]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
)


array([0.01493702])

In [60]:
df_pd = df.to_pandas()
df_pd["levenshtein_dist"] = pds2.str_leven(df_pd["s1"], df_pd["s2"])
df_pd.head()

,actual,predicted,0-2,0-9,s1,s2,levenshtein_dist
0,0.0,0.022554,0,1,ib,7,2
1,1.0,0.968186,0,9,FK,1O,2
2,0.0,0.766648,2,3,n,g,1
3,1.0,0.526669,1,5,tX,PX,1
4,1.0,0.341943,2,6,J,fB,2


In [61]:
# If you are using Narwhals, well, Narwhal expressions are not Polars expressions.
# Using the pds2 module, you can run pds functions in map_batches, but this is limited to 1 input column.

import narwhals as nw

df_nw = nw.from_native(df_pd)
df_nw.with_columns(
    nw_levenshtein_dist = nw.col("s1").map_batches(lambda s: pds2.str_leven(s.to_numpy(), pl.lit('k9')))
).head()

┌─────────────────────────────────────────────────────────────────────────────┐
|                             Narwhals DataFrame                              |
|-----------------------------------------------------------------------------|
|   actual  predicted  0-2  0-9  s1  s2  levenshtein_dist  nw_levenshtein_dist|
|0     0.0   0.022554    0    1  ib   7                 2                    2|
|1     1.0   0.968186    0    9  FK  1O                 2                    2|
|2     0.0   0.766648    2    3   n   g                 1                    2|
|3     1.0   0.526669    1    5  tX  PX                 1                    2|
|4     1.0   0.341943    2    6   J  fB                 2                    2|
└─────────────────────────────────────────────────────────────────────────────┘